In [293]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  # or any other model you prefer
from sklearn import svm 
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier  # For classification tasks
from sklearn.neighbors import KNeighborsRegressor   # For regression tasks
import datetime

#### Create JSON file for ingestion

In [269]:
import gspread
import pandas as pd
import json
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

spreadsheet = client.open("Model")
gas = spreadsheet.worksheet("Sheet1")
electric = spreadsheet.worksheet("Sheet2")

gas = gas.get_all_records()
electric = electric.get_all_records()

json_gas = json.dumps(gas, indent=4)
json_electric = json.dumps(electric, indent=4)

with open("gas.json", "w") as json_file:
    json_file.write(json_gas)

with open("electric.json", "w") as json_file:
    json_file.write(json_electric)

#### Import files and split up training and test

In [301]:
import json

current_year = datetime.datetime.now().year

file_gas= './gas.json'

with open(file_gas, 'r') as f:
        consumptions = json.load(f)

df_gas = pd.DataFrame(consumptions)

df_gas_curr = df_gas[df_gas['year'] == current_year]
X1curr = df_gas_curr.drop('consumption',axis=1)
y1curr = df_gas_curr['consumption']

df_gas = df_gas[df_gas['year'] != current_year]

X1 = df_gas.drop('consumption',axis=1)
y1 = df_gas['consumption']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [309]:
file_electric = './electric.json'

with open(file_electric, 'r') as f:
        consumptions = json.load(f) 

df_elec = pd.DataFrame(consumptions)

df_elec_curr = df_elec[df_elec['year'] == current_year]

X2curr = df_elec_curr.drop('consumption',axis=1)
y2curr = df_elec_curr['consumption']

df_elec = df_elec[df_elec['year'] != current_year]

X2 = df_elec.drop('consumption',axis=1)
y2 = df_elec['consumption']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

#### Train the models on the gas and electric data

In [285]:
model1 =  RandomForestRegressor(n_estimators=50, random_state=67)
model1.fit(X1_train, y1_train)

y1_pred = model1.predict(X1_test)

mae1 = mean_absolute_error(y1_test, y1_pred)
print(f'Mean Absolute Error: {mae1}')

Mean Absolute Error: 4697.362727272728


In [286]:
model2 =  RandomForestRegressor(n_estimators=50, random_state=67)
model2.fit(X2_train, y2_train)

y2_pred = model2.predict(X2_test)

mae2 = mean_absolute_error(y2_test, y2_pred)
print(f'Mean Absolute Error: {mae2}')

Mean Absolute Error: 130054.2918181818


#### Print model score and show comparisons

In [323]:
print(model1.score(X1_train, y1_train))

results = pd.DataFrame({
    'Actual Consumption': y1_test,
    'Predicted Consumption': y1_pred
})

#print(results)

0.9705703672153084


In [324]:
print(model2.score(X2_train, y2_train))

results = pd.DataFrame({
    'Actual Consumption': y2_test,
    'Predicted Consumption': y2_pred
})

#print(results)

0.9508246458554248


In [327]:
pred_gas_curr = model1.predict(X1curr)
pred_elec_curr = model2.predict(X2curr)

curr_results = pd.DataFrame({
    'Gas Prediction': pred_gas_curr,
    'Gas Consumption': y1curr,
    'Electric Prediction': pred_elec_curr,
    'Electric Consumption': y2curr
})

#print(curr_results)
try:
    pred_worksheet = spreadsheet.add_worksheet(title="Predictions", rows="100", cols="4")
except gspread.exceptions.APIError:
    pred_worksheet = spreadsheet.worksheet("Predictions")

pred_data = [curr_results.columns.values.tolist()] + curr_results.values.tolist()

pred_worksheet.update(range_name='C1', values=pred_data)

{'spreadsheetId': '17gvUngyBcr26ZqU4KmNK3oG6XyPKDdybpNg8nl5ioIA',
 'updatedRange': 'Predictions!C1:F7',
 'updatedRows': 7,
 'updatedColumns': 4,
 'updatedCells': 28}